# SAP Cloud ALM Project and Task API Demo for Timeboxes, Task List Queries, Sub-Tasks, Comments, References, New Attributes

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for Implementation. The code cells below show how to use the Project and Task APIs with the latest updates as of **2022-02-01**.

### New Features

* Project Timeboxes: List, Create, Read, Update Delete
* Task List Queries: offset, limit, type, status, subStatus, assigneeId, lastChangedDate 
* Sub-tasks: List, Create, Read, Update Delete
* Comments: List, Create, Read, Update Delete
* References (URLs): List, Create, Read, Update Delete
* New Task Attributes: assignedRole, scope, storyPoints, timebox

The API information and specification is available here:

* https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
* https://api.sap.com/api/CALM_PJM/overview - SAP Cloud ALM Projects
* https://api.sap.com/api/CALM_TKM/overview - SAP Cloud ALM Tasks

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples

### Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

* Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support 
* Python 3, a recent version, is Required. Python 3.8 was used here.
* Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://docs.python-requests.org/en/latest/user/install/#install
* Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
* Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html
* Plotly - for plotting interactive charts - https://plotly.com/python/getting-started/



---

## Authentication information

You must create a python module file called `apidata.py` and put the information specific to your tenant there. This inclduds

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is senstive.

These items can be retrieved from the BTP Cockpit 

### Format of module apidata.py for import

```python
ptm_all_client_id = r'get your client ID from BTP Cockpit'
ptm_all_client_secret = r'get your client secret from BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```


In [ ]:
import apidata as ad

client_id = ad.ptm_all_client_id
client_secret = ad.ptm_all_client_secret
token_url = ad.token_url
base_url = ad.base_url

### Get token for authentication

Call OAuth token API with credential information. Add the resulting header to all requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

hed = {'Authorization': 'Bearer ' + token['access_token']}        


---

## Project APIs

The following code finds a project which we'll use below for the actual code samples.

### Perform GET request to retrieve list of all projects

Expected response: `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-projects/v1/projects', headers=hed)

print(response.status_code, response.reason)

### Parse JSON into Pandas Dataframe

This takes the data returned from the projects API, which is in JSON format, and places it into a dataframe for further processing and analysis.

In [ ]:
import pandas as pd

df = pd.json_normalize(response.json())

df

### Get a Project Name 

You may wish to adjust the code below to set a project name of your choosing, for example 'My API Test Project'. In this sample, we'll just take the first project in the list.


In [ ]:
sample_project_name = df.iloc[0]['name'] # First project in the list
sample_project_name

#### Override Sample Project Name for Demo

I'm setting my own project name here, beause I have one ready for my demos. You can do the same if you don't want to use the first project.

In [ ]:

# sample_project_name = 'Demo Project'

### Get ID of a Selected Project

Read ID of project in variable `sample_project_name` from dataframe

In [ ]:
stPrjID = df.loc[df['name'] == sample_project_name]
show_tell_prj = stPrjID['id'].values[0]
print('Project:', stPrjID['name'].values[0], 'ID:', show_tell_prj)


---

## Read *Filtered* Task list from selected project

Use GET request from tasks API with selected project ID as input parameter.

**New 2022-02-01** filter parameters to limit task list. The code below shows a few of the filters. See the documentation for the full list.

Expected response `200 OK`

In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks?projectId=' + show_tell_prj

#taskURL += '&type=CALMUS' # Filter for user stories only 
#taskURL += '&assigneeId=email@example.com' # Filter for assignee only
taskURL += '&lastChangedDate=gt:2022-02-01' # Last changed date greater than

response = requests.get(taskURL, headers=hed)

print(response.status_code, response.reason)

df_stTasks = pd.json_normalize(response.json())

df_stTasks.head()


---

## Project Timeboxes

**New 2022-02-01** list project timeboxes, create, read, update, delete

### Get Timeboxes From Project

Use GET request from projects API with selected project ID as path selector.

Type | Timebox
---- | -----
0    | Milestone
1    | Phase
2    | Sprint

Endpoint: `https://<tenant url>/api/calm-projects/v1/projects/{id}/timeboxes`

Expected response `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-projects/v1/projects/' + show_tell_prj + '/timeboxes', headers=hed)

print(response.status_code, response.reason)

df_timeboxes = pd.json_normalize(response.json())

df_timeboxes.head()

### Create a Timebox

Use POST request from projects API with selected project ID as path selector.

Endpoint: `https://<tenant url>/api/calm-projects/v1/projects/{id}/timeboxes`

Expected response `201 Created`

In [ ]:
import datetime

isoNow = datetime.datetime.now().isoformat()

timeboxCreateURL = base_url + '/api/calm-projects/v1/projects/' + show_tell_prj + '/timeboxes'

timeboxData = {
    "type": 2,  # Make timebox a sprint
    "name": "API Demo Sprint " + isoNow,
    "startDate": "2022-02-07",
    "endDate": "2022-02-18",
    "closed": False,
}

response = requests.post(timeboxCreateURL, headers=hed, json=timeboxData)

print(response.status_code, response.reason)

newTimeboxID = response.json()['id']
print("New Timebox (Sprint) ID:", newTimeboxID)



### Update a Timebox

Close the timebox. Setting closed to True deactivates the sprint, removing it from the list of timeboxes which users can assign to tasks. (Closed timeboxes remain available as filters, just not for assigning.)

Expected response: `200 OK`

In [ ]:
timeboxUpdateURL = base_url + '/api/calm-projects/v1/timeboxes/' + newTimeboxID

timeboxData = {
    "closed": True,
}

response = requests.patch(timeboxUpdateURL, headers=hed, json=timeboxData)

print(response.status_code, response.reason)



---

## Create Parent Task (User Story) to Host Sub-Tasks

* Endpoint: `https://<tenant url>/api/calm-tasks/v1/tasks`

* Type: Post

Replace `email@example.com` with a valid e-mail address.

Expected response `201 Created`


In [ ]:
isoNow = datetime.datetime.now().isoformat()

taskCreateURL = base_url + '/api/calm-tasks/v1/tasks'

taskData = {
    "projectId": show_tell_prj,
    "title": "API Demo User Story with Sub-Tasks " + isoNow, # Append a timestamp to the end of the new task title for easy identification in UI.
    "type": "CALMUS", # Make task type a User Story
    "description": "test description. hello world",
    "assigneeId": "email@example.com",
    "storyPoints": 5
}

response = requests.post(taskCreateURL, headers=hed, json=taskData)

print(response.status_code, response.reason)

newTaskID = response.json()['id']
print("New Task (User Story) ID:", newTaskID)


---

## Create Sub-task

* Endpoint: `https://<tenant url>/api/calm-tasks/v1/tasks`

* Type: Post

The following code creates five sub-tasks, all assigned to the new User Story created above.

Expected response `201 Created`

Sub-tasks have the mostly the same attributes as tasks. The following attributes **must** be set for sub-tasks:

`type`: `CALMST` 

`parentId`: UUID of parent task - this is **mandatory**.

In [ ]:
import datetime

taskCreateURL = base_url + '/api/calm-tasks/v1/tasks'

romNums = ['i', 'ii', 'iii ', 'iv', 'v']
for romIterator in romNums:

    isoNow = datetime.datetime.now().isoformat()

    taskData = {
        "projectId": show_tell_prj,
        "title": romIterator + " API Test Sub-Task " + isoNow,
        "type": "CALMST", # A Sub-Task
        "description": "test description. hello world",
        "assigneeId": "email@example.com",
        "parentId": newTaskID
    }

    response = requests.post(taskURL, headers=hed, json=taskData)

    print(response.status_code, response.reason)

    newSubTaskID = response.json()['id']
    print(romIterator, "New Sub-task ID:", newSubTaskID)


---

## Comments

**New 2022-02-01:** Comments in Tasks. 

* Endpoint pattern: `https://<tenant url>/api/calm-tasks/v1/tasks/{taskId}/comments`

* Types: GET/POST/PATCH/DELETE

These are the text comments which can be added to tasks. Update and delete operations behave exactly as demonstrated below with references.


### Add a Comment to an Existing Task

Expected response: `201 Created`

In [ ]:
commentCreateURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID + '/comments'

isoNow = datetime.datetime.now().isoformat()

commentData = {
    "text": "This is a new comment. Timestamp: " + isoNow
}

response = requests.post(commentCreateURL, headers=hed, json=commentData)

print(response.status_code, response.reason)

newCommentID = response.json()['id']
print("New comment ID:", newCommentID)

### Read Comments From Task

Expected response: `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-tasks/v1/tasks/' + newTaskID + '/comments', headers=hed)

print(response.status_code, response.reason)

df_comments = pd.json_normalize(response.json())

df_comments.head()


---

## References

**New 2022-02-01:** References in Tasks. 

* Endpoint pattern: `https://<tenant url>/api/calm-tasks/v1/tasks/{taskId}/references`

* Types: GET/POST/PATCH/DELETE

These are URLs - links to documents - which can be attached to tasks. The usual create/read/update/delete operations are supported.

### Add Reference to Existing Task

Expected response: `201 Created`

In [ ]:
referenceCreateURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID + '/references'

isoNow = datetime.datetime.now().isoformat()

referenceData = {
    "name": "Link to SAP: " + isoNow,
    "url": "http://www.sap.com"
}

response = requests.post(referenceCreateURL, headers=hed, json=referenceData)

print(response.status_code, response.reason)

newReferenceID = response.json()['id']
print("New reference ID:", newReferenceID)

### Update A Reference

Expected response: `200 OK`

In [ ]:
referenceUpdateURL = base_url + '/api/calm-tasks/v1/tasks/' + 'references/' + newReferenceID

isoNow = datetime.datetime.now().isoformat()

referenceData = {
    "name": "Expert Portal for Implementation: " + isoNow,
    "url": "https://support.sap.com/en/alm/sap-cloud-alm/implementation/sap-cloud-alm-implementation-expert-portal.html"
}

response = requests.patch(referenceUpdateURL, headers=hed, json=referenceData)

print(response.status_code, response.reason)

newCommentID = response.json()['id']
print("New reference ID:", newCommentID)

### Read References from Task

Expected response: `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-tasks/v1/tasks/' + newTaskID + '/references', headers=hed)

print(response.status_code, response.reason)

df_references = pd.json_normalize(response.json())

df_references.head()

### Delete a Reference

Expected response: `204 No Content`

In [ ]:
referenceUpdateURL = base_url + '/api/calm-tasks/v1/tasks/' + 'references/' + newReferenceID

response = requests.delete(referenceUpdateURL, headers=hed)

print(response.status_code, response.reason)


### Re-Read References

Expected response: `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-tasks/v1/tasks/' + newTaskID + '/references', headers=hed)

print(response.status_code, response.reason)

df_references = pd.json_normalize(response.json())

df_references.head()

---

## Further Task Operations


### Read Task

Read the details of the task to check the `obsolete` attribute. (Rerun this cell after updates to check the changed attributes)

Expected response: `200 OK`

In [ ]:
response = requests.get(base_url + '/api/calm-tasks/v1/tasks/' + newTaskID, headers=hed)

print(response.status_code, response.reason)

response.json()

### Update Task - Make Obsolete

**New 2022-02-01:** Obsolete Attribute in API. 

It is now the recommended procedure to set tasks to obsolete rather than directly deleting them. After a waiting period, to ensure that the task is no longer needed, delete tasks.

Expected response: `200 OK`

In [ ]:
taskUpdateURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID

taskUpdateData = {
    "obsolete": True,
}

response = requests.patch(taskUpdateURL, headers=hed, json=taskUpdateData)

print(response.status_code, response.reason)



### Delete Task

Although the API can directly delete tasks, we don't recommend this. Instead we suggest using obsolete as a recycle bin, with a period of grace - several days or weeks - before permanently (and irretrievably) deleting tasks.

Expected response: `204 No Content`

In [ ]:
response = requests.delete(base_url + '/api/calm-tasks/v1/tasks/' + newTaskID, headers=hed)

print(response.status_code, response.reason)